# C-Town year simulation using weeks

This notebook can be used to run a C-Town simulation for 9 months in a "week-by-week" (weeks of 10-days) basis
The notebook reads from .csv files to define the tank level initial conditions for the week and the demand patterns for that week. 
The tank initial levels are stored in "tank_initial_conditions" and the demand points are defined in "starting_demand_points" and "three_years_demands_ctown"

In [2]:
# load module
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import wntr
import wntr_utils
from sklearn.preprocessing import MinMaxScaler
from random import shuffle
import csv
import random
import math

In [17]:
radius_cm = 10.663381187156988
radius_in = radius_cm/2.54
side_in = 30
box_volume_in = math.pow(side_in, 3)
ball_volume_in = (4/3)*math.pow(radius_in, 3)*math.pi
balls = box_volume_in/ball_volume_in
print(balls)

87.1147112976129


In [9]:
67/(2*math.pi)

10.663381187156988

In [2]:
week_index = 0
total_demands = pd.read_csv('three_year_demands_ctown.csv', index_col=0)
demand_starting_points = pd.read_csv('starting_demand_points.csv', index_col=0)
initial_tank_levels = pd.read_csv('tank_initial_conditions.csv', index_col=0)
network_delays = pd.read_csv('network_links_delay.csv', index_col=0)
network_losses = pd.read_csv('network_loss_small.csv', index_col=0)

In [3]:
# load EPANET model
inp_file = 'ctown_map_with_controls.inp'
wn = wntr.network.WaterNetworkModel(inp_file)

C:\Users\Azulm\Anaconda3\lib\site-packages\wntr\epanet\io.py:2085: UserWarning: Not all curves were used in "ctown_map_with_controls.inp"; added with type None, units conversion left to user
  warnings.warn('Not all curves were used in "{}"; added with type None, units conversion left to user'.format(self.wn.name))


In [4]:
wn.options.time.duration

864000

In [5]:
wn.options.time.duration

864000

In [6]:
wn.options.time.hydraulic_timestep

300

In [4]:
week_index = 0
week_start = demand_starting_points.iloc[week_index][0]

In [5]:
week_demands = total_demands.loc[week_start:week_start+240:]

In [6]:
# Replace the demand patterns with the random patterns for this week
names, patterns =[],[]
for name, pat in wn.patterns():  
    pat.multipliers = week_demands[name].values.tolist()

In [11]:
wn.options.time.duration

864000

In [12]:
for pat in wn.patterns():  
    print(pat)

('DMA1_pat', <Pattern 'DMA1_pat', multipliers=array([0.819, 0.633, 0.634, 0.639, 0.538, 0.491, 0.565, 0.555, 0.536, 0.602, 0.567, 0.512, 0.528, 0.519, 0.384, 0.482, 0.426, 0.35 , 0.327, 0.394, 0.36 , 0.623, 0.691, 0.643, 0.709, 0.676, 0.643, 0.608, 0.566, 0.553, 0.504, 0.538, 0.558, 0.547, 0.559, 0.587, 0.539, 0.466, 0.438, 0.479, 0.486,
       0.335, 0.345, 0.337, 0.391, 0.494, 0.703, 0.693, 0.693, 0.738, 0.565, 0.641, 0.599, 0.533, 0.534, 0.536, 0.538, 0.547, 0.566, 0.599, 0.537, 0.48 , 0.468, 0.539, 0.42 , 0.341, 0.4  , 0.367, 0.405, 0.481, 0.625, 0.781, 0.731, 0.7  , 0.613, 0.596, 0.633, 0.596, 0.49 , 0.494, 0.566, 0.543,
       0.514, 0.585, 0.492, 0.502, 0.457, 0.624, 0.475, 0.325, 0.346, 0.339, 0.42 , 0.522, 0.643, 0.772, 0.727, 0.723, 0.644, 0.613, 0.575, 0.583, 0.546, 0.556, 0.6  , 0.63 , 0.553, 0.57 , 0.522, 0.465, 0.443, 0.606, 0.473, 0.354, 0.329, 0.366, 0.427, 0.504, 0.702, 0.727, 0.754, 0.723, 0.612,
       0.594, 0.554, 0.538, 0.587, 0.508, 0.606, 0.627, 0.542, 0.518, 0.

In [7]:
# Replace the initial tank levels with the values stored in the .csv file
for i in range(1,8):    
    wn.get_node('T' + str(i)).init_level = float(initial_tank_levels.iloc[week_index]['T'+ str(i)])

In [8]:
initial_tank_levels.iloc[week_index]

T1    5.622006
T2    1.373781
T3    2.109871
T4    3.680605
T5    1.466813
T6    1.915251
T7    4.015033
Name: 0, dtype: float64

In [13]:
# load EPANET model
inp_file = 'ctown_year_simulation_week_0.inp'
wn = wntr.network.WaterNetworkModel(inp_file)

C:\Users\Azulm\Anaconda3\lib\site-packages\wntr\epanet\io.py:2085: UserWarning: Not all curves were used in "ctown_year_simulation_week_0.inp"; added with type None, units conversion left to user
  warnings.warn('Not all curves were used in "{}"; added with type None, units conversion left to user'.format(self.wn.name))


In [9]:
wn.write_inpfile('ctown_year_simulation_week_0.inp')

In [14]:
# Simulate hydraulics
sim = wntr.sim.WNTRSimulator(wn,mode='PDD')

In [15]:
# this would run the simulation. 
# Mohammad, using this notebook as base you could change the notebook to run all the 38 weeks. Notice that the variable week_index 
# is the one controlling which week is being simulated
results_new = sim.run_sim()

In [18]:
results_new.node['pressure']

,J511,J411,J414,J415,J416,J417,J418,J419,J310,J311,...,J317,J323,T1,T2,T3,T4,T5,T6,T7,R1
0,29.652194,63.341168,37.908214,60.007658,82.407658,34.887165,28.072759,25.196053,48.081994,44.004845,...,54.462816,54.462816,5.622006,1.373781,2.109871,3.680605,1.466813,1.915251,4.015033,0.0
300,29.505340,63.293808,37.860854,59.864337,82.264337,34.839806,28.043425,25.166395,47.928670,43.851522,...,54.328815,54.328815,5.572974,1.343181,2.151765,3.533305,1.501426,1.884452,3.832411,0.0
600,29.358501,63.246458,37.813504,59.720958,82.120958,34.792456,28.014112,25.136758,47.775623,43.698475,...,54.194878,54.194878,5.523952,1.312604,2.193606,3.386029,1.535969,1.853355,3.650261,0.0
900,29.211687,63.199118,37.766164,59.577617,81.977617,34.745115,27.984821,25.107144,47.622859,43.545710,...,54.061015,54.061015,5.474941,1.282050,2.235394,3.238777,1.570442,1.821961,3.468579,0.0
1200,29.064898,63.151787,37.718833,59.434314,81.834314,34.697785,27.955553,25.077552,47.470204,43.393055,...,53.926937,53.926937,5.425939,1.251519,2.277130,3.091548,1.604846,1.790273,3.287367,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
862800,30.629164,62.979250,36.154683,84.831988,98.828903,32.708184,32.498074,29.297357,53.191002,49.110833,...,66.934546,66.934546,2.057739,4.809672,3.716596,3.212895,2.528802,5.291208,2.667536,0.0
863100,30.622502,62.959570,36.144458,84.825721,98.822446,32.701000,32.492366,29.292734,53.190953,49.110784,...,66.935985,66.935985,2.027074,4.806158,3.683869,3.206230,2.555107,5.302550,2.667220,0.0
863400,30.615841,62.939876,36.134094,84.819405,98.815964,32.693636,32.486457,29.287912,53.190920,49.110750,...,66.937425,66.937425,1.996503,4.802476,3.651142,3.199568,2.581371,5.313850,2.666923,0.0
863700,30.609181,62.920168,36.123593,84.813041,98.809458,32.686096,32.480352,29.282895,53.190901,49.110732,...,66.938866,66.938866,1.966024,4.798629,3.618414,3.192906,2.607595,5.325108,2.666643,0.0


In [19]:
pressure_results = results_new.node['pressure']
pressure_results.to_csv('test_ctown_week_0_wntr.csv')